# 情感分析

本notebook参考了https://github.com/bentrevett/pytorch-sentiment-analysis

在这份notebook中，我们会用PyTorch模型和TorchText再来做情感分析(检测一段文字的情感是正面的还是负面的)。我们会使用[IMDb 数据集](http://ai.stanford.edu/~amaas/data/sentiment/)，即电影评论。

模型从简单到复杂，我们会依次构建：
- Word Averaging模型
- RNN/LSTM模型
- CNN模型

## 准备数据

- TorchText中的一个重要概念是`Field`。`Field`决定了你的数据会被怎样处理。在我们的情感分类任务中，我们所需要接触到的数据有文本字符串和两种情感，"pos"或者"neg"。
- `Field`的参数制定了数据会被怎样处理。
- 我们使用`TEXT` field来定义如何处理电影评论，使用`LABEL` field来处理两个情感类别。
- 我们的`TEXT` field带有`tokenize='spacy'`，这表示我们会用[spaCy](https://spacy.io) tokenizer来tokenize英文句子。如果我们不特别声明`tokenize`这个参数，那么默认的分词方法是使用空格。
- 安装spaCy
```
pip install -U spacy
python -m spacy download en
```
- `LABEL`由`LabelField`定义。这是一种特别的用来处理label的`Field`。我们后面会解释dtype。
- 更多关于`Fields`，参见https://github.com/pytorch/text/blob/master/torchtext/data/field.py
- 和之前一样，我们会设定random seeds使实验可以复现。

In [1]:
import torch
from torchtext import data

import en_core_web_sm
import spacy
spacy.load('en_core_web_sm')

USE_CUDA = torch.cuda.is_available()
SEED = 1234

#random.seed(SEED)
torch.manual_seed(SEED)
if USE_CUDA:
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.daterministic = True
TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)
    


- TorchText支持很多常见的自然语言处理数据集。
- 下面的代码会自动下载IMDb数据集，然后分成train/test两个`torchtext.datasets`类别。数据被前面的`Fields`处理。IMDb数据集一共有50000电影评论，每个评论都被标注为正面的或负面的。

In [2]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


.data\imdb\aclImdb_v1.tar.gz: 100%|█████| 84.1M/84.1M [1:07:30<00:00, 26.2kB/s]


查看每个数据split有多少条数据。

In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


查看一个example。

In [5]:
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

- 由于我们现在只有train/test这两个分类，所以我们需要创建一个新的validation set。我们可以使用`.split()`创建新的分类。
- 默认的数据分割是 70、30，如果我们声明`split_ratio`，可以改变split之间的比例，`split_ratio=0.8`表示80%的数据是训练集，20%是验证集。
- 我们还声明`random_state`这个参数，确保我们每次分割的数据集都是一样的。

In [6]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

检查一下现在每个部分有多少条数据。

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


- 下一步我们需要创建 _vocabulary_ 。_vocabulary_ 就是把每个单词一一映射到一个数字。
![](assets/sentiment5.png)
- 我们使用最常见的25k个单词来构建我们的单词表，用`max_size`这个参数可以做到这一点。
- 所有其他的单词都用`<unk>`来表示。

In [ ]:
''' 创建 vocabulary ，输入参数 文本， 字典最大长度， 使用glove计算的100维的词向量， 未知单词的词向量用normal代替'''
TEXT.build_vocab(train_data, max_size=25000, vectors="glove.6B.100d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

.vector_cache\glove.6B.zip:   2%|       | 15.3M/862M [24:42<16:10:34, 14.5kB/s]

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

- 当我们把句子传进模型的时候，我们是按照一个个 _batch_ 穿进去的，也就是说，我们一次传入了好几个句子，而且每个batch中的句子必须是相同的长度。为了确保句子的长度相同，TorchText会把短的句子pad到和最长的句子等长。
![](assets/sentiment6.png)
- 下面我们来看看训练数据集中最常见的单词。

In [ ]:
print(TEXT.vocab.freqs.most_common(20))

我们可以直接用 `stoi`(**s**tring **to** **i**nt) 或者 `itos` (**i**nt **to**  **s**tring) 来查看我们的单词表。

In [3]:
print(TEXT.vocab.itos[:10])

AttributeError: 'Field' object has no attribute 'vocab'

查看labels。

In [ ]:
print(LABEL.vocab.stoi)

- 最后一步数据的准备是创建iterators。每个itartion都会返回一个batch的examples。
- 我们会使用`BucketIterator`。`BucketIterator`会把长度差不多的句子放到同一个batch中，确保每个batch中不出现太多的padding。
- 严格来说，我们这份notebook中的模型代码都有一个问题，也就是我们把`<pad>`也当做了模型的输入进行训练。更好的做法是在模型中把由`<pad>`产生的输出给消除掉。在这节课中我们简单处理，直接把`<pad>`也用作模型输入了。由于`<pad>`数量不多，模型的效果也不差。
- 如果我们有GPU，还可以指定每个iteration返回的tensor都在GPU上。

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

## Word Averaging模型

- 我们首先介绍一个简单的Word Averaging模型。这个模型非常简单，我们把每个单词都通过`Embedding`层投射成word embedding vector，然后把一句话中的所有word vector做个平均，就是整个句子的vector表示了。接下来把这个sentence vector传入一个`Linear`层，做分类即可。

![](assets/sentiment8.png)

- 我们使用[`avg_pool2d`](https://pytorch.org/docs/stable/nn.html?highlight=avg_pool2d#torch.nn.functional.avg_pool2d)来做average pooling。我们的目标是把sentence length那个维度平均成1，然后保留embedding这个维度。

![](assets/sentiment9.png)

- `avg_pool2d`的kernel size是 (`embedded.shape[1]`, 1)，所以句子长度的那个维度会被压扁。

![](assets/sentiment10.png)

![](assets/sentiment11.png)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WordAVGModel(nn.Module):
    def __init__(self,vocab_size, embed_size, output_size,pad_idx):
        super(WordAVGModel, self).__init__()
        
        # padding_idx 是在 embedding 的过程中将 padding 的单词的词向量初始化为 0
        # pad_idx 是指 pad 单词的 index
        self = nn.Embedding(vocab_size, embed_size, padding_idx = pad_idx)
        self.linear = nn.Linear(embedding_size, output_size)
        
        
    def forward(self, text):
        # text：[sent len, batch size]
        embeded = self.embed(text)# [sent len, batch size, emb dim]
        #embeded =embeded.transpose(1,0) # [batch_size, sequence_len, embedding_dimension]
        embeded = embeded.permute(1,0,2) #[batch_size, sequence_len, embedding_dimension] 重新排序
        
        '''这里之所以能去掉一个 sentence length 维度 是因为 我们直接对 其的 embedding 维度进行取平均 最终每个batch只留下对应个数的embedding'''
        ''' avg_pool2d 压缩 成2维数据 
        torch.nn.functional.avg_pool2d(input, 
                                        kernel_size, 
                                        stride=None, 
                                        padding=0, 
                                        ceil_mode=False,
                                        count_include_pad=True,
                                        divisor_override=None) → Tensor '''
        pooled = F.avg_pool2d(embeded, (embeded.shape(1),1)) # [batch_size, 1, embedding_size]
        pooled = peeled.squeeze() # [batch_size, embedding_size]
        
        return self.linear(pooled)
        

In [ ]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_SIZE = 100
OUTPUT_SIZE = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token] # TEXT.pad_token 实际上就是 <pad>

model = WordAVGModel(vocab_size = VOCAB_SIZE,
                     embed_size = EMBEDDING_SIZE,
                     output_size = OUTPUT_SIZE,
                     pad_idx = PAD_IDX)

In [ ]:
''' 统计模型的参数的数量，有时我们不希望模型的参数太多 '''
def count_parameters(model):
    ''' numel 会自动计算出当前 tensor 中参数的数量'''
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

Glove初始化

In [ ]:
''' 因为我们在创建 TEXT.vocab 时 添加了 vector 数据，我们将 glove 预训练的 100维的词向量，保存到了 vocab 中
    我们使用预训练的 词向量 将 model 的权重初始化。
'''
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
''' 我们将 unk 与 pad 的词向量 变为 0  '''
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## 训练模型

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
''' BCE -> binary cross entorpy
    withLogits -> 不对数据进行 sigmoid 
    BCEwithLongitsLoss 只针对二分类问题进行loss 计算的
    '''
crit = nn.BCEwithLongitsLoss()

model = model.to(device)
crit = crit.to(device)


计算预测的准确率

In [ ]:
def binary_accuracy(preds, y):
    ''' torch.round 四舍五入 将 sigmoid 的结果 四舍五入到 0，1'''
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() # 将比较结果，保存成数值
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model , iterator, optimizer, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.train()
    total_len = 0.
    
    for batch in iterator:
        preds = model(batch.text).squeeze()
        loss = (preds, batch.label)
        acc = binary_accuracy(preds, batch.label)
        
        # sgd
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() * len(batch.label)
        epoch_acc += acc.item() * len(batch.label)
        total_len += len(batch.label)
    return epoch_loss / total_len , epoch_acc / total_len

In [ ]:
''' evaluate 和 train 很像，只是去掉的 train 中 进行参数优化的部分'''
def evaluate(model, iterator, crit):
    epoch_loss, epoch_acc = 0., 0.
    model.eval()
    total_len = 0.
    with torch.no_grad():
        for batch in iterator:
            preds = model(batch.text).squeeze()
            loss = (preds, batch.label)
            acc = binary_accuracy(preds, batch.label)


            epoch_loss += loss.item() * len(batch.label)
            epoch_acc += acc.item() * len(batch.label)
            total_len += len(batch.label)
    model.train()
    return epoch_loss / total_len , epoch_acc / total_len

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
NUM_EPOCHS = 10
best_valid_loss = 0.
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, crit )
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit )
    
    if valid_acc > best_valid_loss:
        best_valid_loss = valid_acc
        torch.save(model.state_dict(),'WordAVG-Model.pth')
        
    print('Epoch: ',epoch,' Train Loss: ',train_loss," Trian Acc: ",train_acc)
    print("Epoch:",epoch, " Valid Loss: ",valid_loss, " Valid Acc: ",valid_acc)

测试模型

In [ ]:
model.load_state_dict(torch.load("WordAVG-Model.pth"))

In [ ]:
import spacy
nlp = spacy.load("en")

def predict_sentiment(sentence):
    ''' 先将文本进行预处理（分词）'''
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    ''' 将文本进行数字化'''
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    ''' 将文本转换为 long tensor 类型'''
    tensor = torch.LongTensor(indexed).to(device) # seq_len
    '''  将文本转换为 sequence length * batch size 形状'''
    tensor = tensor.unsqueeze(1) # seq_len * batch_size(1)
    pred = torch.sigmoid(model(tensor))
    return pred.item()

In [ ]:
predict_sentiment("This film is horrible!")

In [ ]:
predict_sentiment("This film is terrific!")

## RNN模型

- 下面我们尝试把模型换成一个**recurrent neural network** (RNN)。RNN经常会被用来encode一个sequence
$$h_t = \text{RNN}(x_t, h_{t-1})$$
- 我们使用最后一个hidden state $h_T$来表示整个句子。
- 然后我们把$h_T$通过一个线性变换$f$，然后用来预测句子的情感。

![](assets/sentiment1.png)

![](assets/sentiment7.png)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, pad_idx, hidden_size, output_size, dropout):
        super(RNNModel, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_size, padding_idx = pad_idx)
        
        self.lstm = nn.LSTM(embedding_size, hidden_size, bidirectional =True, num_layers = 2)
        self.linear = nn.Linear(hidden_size*2, output_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        embeded = self.embed(text)
        embeded = self.dropour(embeded)
        output, (hidden, cell)= self.lstm(embeded)
        
        # 最后一个hidden 是整个句子的表示 【2， batch_size, hidden_size】
        #hidden = torch.cat(hidden[-1],hidden[-2],dim=1)
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        hidden = torch.cat(hidden[-2],hidden[-1],dim=1)    
        hidden = self.dropout(hidden)
        return self.linear(hidden)
        

In [ ]:
model = RNNModel(vocab_size= VOCAB_SIZE,
                 embedding_size=EMBEDDING_SIZE,
                 pad_idx=PAD_SIZE,
                 hidden_size=100,
                 output_size=,
                 dropout=0.5)

In [ ]:
''' 使用 glove 初始化 模型参数'''
pretrained_embedding = TEXT.vocab.vectors
model.embed.weight.data.copy_(pretrained_embedding)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embed.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_SIZE)
model.embed.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_SIZE)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
''' BCE -> binary cross entorpy
    withLogits -> 不对数据进行 sigmoid 
    BCEwithLongitsLoss 只针对二分类问题进行loss 计算的
    '''
crit = nn.BCEwithLongitsLoss()

model = model.to(device)
crit = crit.to(device)


In [ ]:
NUM_EPOCHS = 10
best_valid_loss = 0.
for epoch in range(NUM_EPOCHS):
    train_loss, train_acc = train(model, train_iterator, optimizer, crit )
    valid_loss, valid_acc = evaluate(model, valid_iterator, crit )
    
    if valid_acc > best_valid_loss:
        best_valid_loss = valid_acc
        torch.save(model.state_dict(),'LSTM-Model.pth')
        
    print('Epoch: ',epoch,' Train Loss: ',train_loss," Trian Acc: ",train_acc)
    print("Epoch:",epoch, " Valid Loss: ",valid_loss, " Valid Acc: ",valid_acc)

## 训练RNN模型

You may have noticed the loss is not really decreasing and the accuracy is poor. This is due to several issues with the model which we'll improve in the next notebook.

Finally, the metric we actually care about, the test loss and accuracy, which we get from our parameters that gave us the best validation loss.

## CNN模型

In [ ]:

class CNN(nn.Module):
    def __init__(self,vocab_size, embed_size, output_size,pad_idx):
        super(CNN, self).__init__()
        
        # padding_idx 是在 embedding 的过程中将 padding 的单词的词向量初始化为 0
        # pad_idx 是指 pad 单词的 index
        self = nn.Embedding(vocab_size, embed_size, padding_idx = pad_idx)
        self.linear = nn.Linear(embedding_size, output_size)
        
        
    def forward(self, text):
        # text：[sent len, batch size]
        embeded = self.embed(text)# [sent len, batch size, emb dim]
        
        
        return self.linear(pooled)